# #***Data loading rerec***

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed

# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# For example, here's several helpful packages to load



import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



# Input data files are available in the read-only "../input/" directory

# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



import os

real_rerec_audio=[]

fake_rerec_audio=[]

# fake_audio_path = ['/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/fake','/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/fake','/kaggle/input/the-fake-or-real-dataset/for-original/for-original/training/fake','/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/fake']

# real_audio_path = ['/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/real','/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/real','/kaggle/input/the-fake-or-real-dataset/for-original/for-original/training/real','/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/real']





fake_rerecaudio_path='/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/fake'

real_rerecaudio_path='/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/real'

# for _dir in fake_audio_path:

for filename in os.listdir(fake_rerecaudio_path):

    if filename.endswith('.wav'):

        fake_rerec_audio.append(os.path.join(fake_rerecaudio_path, filename))

#         print(filename)

            

# for _dir in real_audio_path:

for filename in os.listdir(real_rerecaudio_path):

    if filename.endswith('.wav'):

        real_rerec_audio.append(os.path.join(real_rerecaudio_path, filename))



# real_audio_files = [os.path.join(real_audio_path, file) for file in os.listdir(real_audio_path) if file.endswith('.wav')]

# fake_audio_files = [os.path.join(fake_audio_path, file) for file in os.listdir(fake_audio_path) if file.endswith('.wav')]



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import librosa

import matplotlib.pyplot as plt

import seaborn as sns

from tqdm import tqdm

import IPython

from librosa import display

from IPython import display as ipd

In [ ]:
print("Number of tuples in fake_audio dataset: ", len(fake_rerec_audio))
print("Number of tuples in real_audio dataset: ", len(real_rerec_audio))


***For Rerecorded Audio***

> Visualise data

In [ ]:
import os

def remove_zero_byte_files(folder):
    # Loop through all files in the specified directory
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        
        # Check if it's a file and if its size is 0 bytes
        if os.path.isfile(file_path) and os.path.getsize(file_path) == 0:
            print(f"Removing {file_path}")
            os.remove(file_path)

        # print("hello")
# Specify the directory containing your audio files
directory_path = real_rerecaudio_path  # Replace with your directory path
remove_zero_byte_files(directory_path)

In [ ]:
import os
import hashlib

def calculate_md5(file_path, chunk_size=4096):
    """Calculates MD5 hash of a file to identify duplicates."""
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(chunk_size), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def remove_duplicates(directory):
    """Remove duplicate files in a directory based on file content."""
    hashes = {}  # Dictionary to store hashes and their corresponding file paths

    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        # Ensure it is a file and not a directory
        if os.path.isfile(file_path):
            # Calculate hash of the file
            file_hash = calculate_md5(file_path)
            
            # Check if the hash already exists in the dictionary
            if file_hash in hashes:
                print(f"Duplicate found and removing: {file_path}")
                os.remove(file_path)
            else:
                hashes[file_hash] = file_path

    print("Finished removing duplicate files.")

# Usage
directory = real_rerecaudio_path  # Replace with the actual path
remove_duplicates(directory)


In [ ]:
!pip install librosa scikit-learn

In [ ]:
import os
import numpy as np
import librosa
from sklearn.preprocessing import StandardScaler

def preprocess_audio(file_path, target_sample_rate=16000, target_length=16000):
    """Load, resample, pad, and normalize audio data."""
    # Load audio with librosa at the target sample rate
    audio, sample_rate = librosa.load(file_path, sr=target_sample_rate)
    
    # Zero-padding or truncation to match the target length
    if len(audio) < target_length:
        audio = np.pad(audio, (0, target_length - len(audio)), 'constant')
    else:
        audio = audio[:target_length]
    
    return audio

def standardize_audio_data(directory):
    """Process all audio files in a directory and normalize data."""
    audio_data = []
    
    # Process each audio file in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        if os.path.isfile(file_path) and filename.lower().endswith(('.wav', '.mp3')):
            audio = preprocess_audio(file_path)
            audio_data.append(audio)
    
    # Convert to numpy array for batch processing
    audio_data = np.array(audio_data)
    
    # Apply StandardScaler normalization
    scaler = StandardScaler()
    audio_data_normalized = scaler.fit_transform(audio_data)
    
    return audio_data_normalized

# Usage
directory = real_rerecaudio_path  # Replace with the actual path
processed_data = standardize_audio_data(directory)

# Now, processed_data contains zero-padded and normalized audio samples ready for model training.


In [ ]:
preprocessed_real_rerec=[]
for filename in os.listdir(real_rerecaudio_path):

    if filename.endswith('.wav'):

        preprocessed_real_rerec.append(os.path.join(real_rerecaudio_path, filename))

In [ ]:
print("Number of tuples in real_audio dataset: ", len(preprocessed_real_rerec))
# print("Number of tuples in real_audio dataset: ", len(real_rerec_audio))


In [ ]:
# loading audio

ipd.Audio('/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/fake/recording1.wav_norm_mono.wav')

In [ ]:
real_ad, real_sr = librosa.load(real_rerec_audio[0])

plt.figure(figsize=(12, 4))

plt.plot(real_ad)

plt.title("Real Audio Data")

plt.show()

In [ ]:
fake_ad, fake_sr = librosa.load(fake_rerec_audio[0])

plt.figure(figsize=(12, 4))

plt.plot(fake_ad)

plt.title("Fake Audio Data")

plt.show()

In [ ]:
real_spec = np.abs(librosa.stft(real_ad))

real_spec = librosa.amplitude_to_db(real_spec, ref=np.max)



plt.figure(figsize=(10, 3))

librosa.display.specshow(real_spec, sr=real_sr, x_axis="time", y_axis="log")

plt.colorbar(format="%+2.0f dB")

plt.title("Real Audio Spectogram")

plt.show()

In [ ]:
fake_spec = np.abs(librosa.stft(fake_ad))

fake_spec = librosa.amplitude_to_db(fake_spec, ref=np.max)

plt.figure(figsize=(10, 3))

librosa.display.specshow(fake_spec, sr=fake_sr, x_axis="time", y_axis="log")

plt.colorbar(format="%+2.0f dB")

plt.title("Fake Audio Spectogram")

plt.show()

fake data

In [ ]:
import os
folders = os.listdir(fake_rerecaudio_path)

# print(folders)'

***preprocessing and feature extraction rerec***

In [ ]:
import os

rerec_fake_data = [] 
labels = []

for file in tqdm(folders):
    file_path = os.path.join(fake_rerecaudio_path, file)
    audio, sample_rate = librosa.load(file_path)        
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_features_scaled = np.mean(mfccs_features.T, axis=0)
    rerec_fake_data.append(mfccs_features_scaled)

#     labels.append(folder)

***saving mfcc features in excel sheet***


In [ ]:
import pandas as pd

df = pd.DataFrame(rerec_fake_data)

# Add column names (optional but recommended)
num_mfccs = rerec_fake_data[0].shape[0]  # Assuming all have the same number of MFCCs
df.columns = [f'MFCC_{i}' for i in range(num_mfccs)]

# Save DataFrame to Excel
excel_file_path = '/kaggle/working/fake_audio.xlsx'

df.to_excel(excel_file_path, index=False)

print(f"Features saved to: {excel_file_path}")

***Real Data***

In [ ]:
import os
realfolders = os.listdir(real_rerecaudio_path)

***Preprocessing and feature Extraction of Real Data***


In [ ]:
import os

rerec_real_data = []
labels = []

for file in tqdm(realfolders):
    file_path = os.path.join(real_rerecaudio_path, file)
    audio, sample_rate = librosa.load(file_path)        
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_features_scaled = np.mean(mfccs_features.T, axis=0)
    rerec_real_data.append(mfccs_features_scaled)

***saving mfcc features in excel sheet***


In [ ]:
import pandas as pd

df = pd.DataFrame(rerec_real_data)

# Add column names (optional but recommended)
num_mfccs = realdata[0].shape[0]  # Assuming all have the same number of MFCCs

df.columns = [f'MFCC_{i}' for i in range(num_mfccs)]

# Save DataFrame to Excel
excel_file_path = '/kaggle/working/real_re_audio.xlsx'

df.to_excel(excel_file_path, index=False)

print(f"Features saved to: {excel_file_path}")

In [ ]:
!ls /kaggle/working

# #***Data loading 2sec***

In [ ]:
import os

real_2sec_audio=[]

fake_2sec_audio=[]

fake_2sec_audio_path='/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/fake'

real_2sec_audio_path='/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/real'

# for _dir in fake_audio_path:

for filename in os.listdir(fake_2sec_audio_path):

#     if filename.endswith('.wav'):

    fake_2sec_audio.append(os.path.join(fake_2sec_audio_path, filename))
    
# for _dir in real_audio_path:

for filename in os.listdir(real_2sec_audio_path):

#     if filename.endswith('.wav'):

    real_2sec_audio.append(os.path.join(real_2sec_audio_path, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# For-2sec dataset

print("Number of tuples in fake_audio dataset: ", len(fake_2sec_audio))

print("Number of tuples in real_audio dataset: ", len(real_2sec_audio))

***Fake 2sec***

In [ ]:
import os
folders_fake_2sec = os.listdir(fake_2sec_audio_path)

***preprocessing and feature extraction fake2sec***

In [ ]:
import os

fake_2sec_data = []

for file in tqdm(folders_fake_2sec):
    file_path = os.path.join(fake_2sec_audio_path, file)
    audio, sample_rate = librosa.load(file_path)        
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_features_scaled = np.mean(mfccs_features.T, axis=0)
    fake_2sec_data.append(mfccs_features_scaled)

In [ ]:
import pandas as pd

df = pd.DataFrame(fake_2sec_data)

# Add column names (optional but recommended)
num_mfccs = fake_2sec_data[0].shape[0]  # Assuming all have the same number of MFCCs

df.columns = [f'MFCC_{i}' for i in range(num_mfccs)]

# Save DataFrame to Excel
excel_file_path = '/kaggle/working/fake_2sec_audio.xlsx'

df.to_excel(excel_file_path, index=False)

print(f"Features saved to: {excel_file_path}")

***Real 2sec***

In [ ]:
import os

folders_real_2sec = os.listdir(real_2sec_audio_path)

***preprocessing and feature extraction real2sec***

In [ ]:
import os

real_2sec_data = []

for file in tqdm(folders_real_2sec):
    file_path = os.path.join(real_2sec_audio_path, file)
    audio, sample_rate = librosa.load(file_path)        
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_features_scaled = np.mean(mfccs_features.T, axis=0)
    real_2sec_data.append(mfccs_features_scaled)

In [ ]:
import pandas as pd

df = pd.DataFrame(real_2sec_data)

# Add column names (optional but recommended)
num_mfccs = real_2sec_data[0].shape[0]  # Assuming all have the same number of MFCCs

df.columns = [f'MFCC_{i}' for i in range(num_mfccs)]

# Save DataFrame to Excel
excel_file_path = '/kaggle/working/real_2sec_audio.xlsx'

df.to_excel(excel_file_path, index=False)

print(f"Features saved to: {excel_file_path}")

# #***Data loading norm***

In [ ]:
import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

real_norm_audio=[]

fake_norm_audio=[]

fake_norm_audio_path='/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/fake'

real_norm_audio_path='/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/real'

# for _dir in fake_audio_path:

for filename in os.listdir(fake_norm_audio_path):

#     if filename.endswith('.wav'):

    fake_norm_audio.append(os.path.join(fake_norm_audio_path, filename))
    
# for _dir in real_audio_path:

for filename in os.listdir(real_norm_audio_path):

#     if filename.endswith('.wav'):

    real_norm_audio.append(os.path.join(real_norm_audio_path, filename))

In [ ]:
# For-norm dataset

print("Number of tuples in fake_audio dataset: ", len(fake_norm_audio))
print("Number of tuples in real_audio dataset: ", len(real_norm_audio))

***Fake norm***

In [ ]:
import os
folders_fake_norm = os.listdir(fake_norm_audio_path)

***preprocessing and feature extraction fakenorm***

In [ ]:
import os

fake_norm_data = []

for file in tqdm(folders_fake_norm):
    file_path = os.path.join(fake_norm_audio_path, file)
    audio, sample_rate = librosa.load(file_path)        
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_features_scaled = np.mean(mfccs_features.T, axis=0)
    fake_norm_data.append(mfccs_features_scaled)

#     labels.append(folder)

In [ ]:
import pandas as pd

df = pd.DataFrame(fake_norm_data)

# Add column names (optional but recommended)
num_mfccs = fake_norm_data[0].shape[0]  # Assuming all have the same number of MFCCs

df.columns = [f'MFCC_{i}' for i in range(num_mfccs)]

# Save DataFrame to Excel
excel_file_path = '/kaggle/working/fake_norm_audio.xlsx'

df.to_excel(excel_file_path, index=False)

print(f"Features saved to: {excel_file_path}")

***Real norm***

In [ ]:
import os

folders_real_norm = os.listdir(real_norm_audio_path)

***preprocessing and feature extraction realnorm***

In [ ]:
import os

real_norm_data = []

for file in tqdm(folders_real_norm):
    file_path = os.path.join(real_norm_audio_path, file)
    audio, sample_rate = librosa.load(file_path)        
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_features_scaled = np.mean(mfccs_features.T, axis=0)
    real_norm_data.append(mfccs_features_scaled)

In [ ]:
import pandas as pd

df = pd.DataFrame(real_norm_data)

# Add column names (optional but recommended)
num_mfccs = real_norm_data[0].shape[0]  # Assuming all have the same number of MFCCs

df.columns = [f'MFCC_{i}' for i in range(num_mfccs)]

# Save DataFrame to Excel
excel_file_path = '/kaggle/working/real_norm_audio.xlsx'

df.to_excel(excel_file_path, index=False)

print(f"Features saved to: {excel_file_path}")

In [ ]:
!ls /kaggle/working

In [ ]:
pip install matplotlib

# #***MFCC Coefficient extraction***

In [ ]:
import librosa
import matplotlib.pyplot as plt

real_ad, real_sr = librosa.load('/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/fake/recording1.wav_norm_mono.wav')
real_mfccs = librosa.feature.mfcc(y=real_ad, sr=real_sr)

plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mfccs, sr=real_sr, x_axis="time")
plt.colorbar()
plt.title("Real Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

In [ ]:
import librosa
import matplotlib.pyplot as plt

real_ad, real_sr = librosa.load('/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/fake/recording1013.wav_norm_mono.wav')
real_mfccs = librosa.feature.mfcc(y=real_ad, sr=real_sr)

plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mfccs, sr=real_sr, x_axis="time")
plt.colorbar()
plt.title("Real Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

In [ ]:
import librosa
import matplotlib.pyplot as plt

real_ad, real_sr = librosa.load('/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/real/recording10006.wav_norm_mono.wav')
real_mfccs = librosa.feature.mfcc(y=real_ad, sr=real_sr)

plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mfccs, sr=real_sr, x_axis="time")
plt.colorbar()
plt.title("Real Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

In [ ]:
import librosa
import matplotlib.pyplot as plt

real_ad, real_sr = librosa.load('/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/real/recording10039.wav_norm_mono.wav')
real_mfccs = librosa.feature.mfcc(y=real_ad, sr=real_sr)

plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mfccs, sr=real_sr, x_axis="time")
plt.colorbar()
plt.title("Real Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

In [ ]:
import librosa
import matplotlib.pyplot as plt

real_ad, real_sr = librosa.load('/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/fake/file10058.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav')
real_mfccs = librosa.feature.mfcc(y=real_ad, sr=real_sr)

plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mfccs, sr=real_sr, x_axis="time")
plt.colorbar()
plt.title("Real Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

In [ ]:
import librosa
import matplotlib.pyplot as plt

real_ad, real_sr = librosa.load('/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/fake/file10286.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav')
real_mfccs = librosa.feature.mfcc(y=real_ad, sr=real_sr)

plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mfccs, sr=real_sr, x_axis="time")
plt.colorbar()
plt.title("Real Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

In [ ]:
import librosa
import matplotlib.pyplot as plt

real_ad, real_sr = librosa.load('/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/real/file10075.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav')
real_mfccs = librosa.feature.mfcc(y=real_ad, sr=real_sr)

plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mfccs, sr=real_sr, x_axis="time")
plt.colorbar()
plt.title("Real Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

In [ ]:
import librosa
import matplotlib.pyplot as plt

real_ad, real_sr = librosa.load('/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/real/file10269.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav')
real_mfccs = librosa.feature.mfcc(y=real_ad, sr=real_sr)

plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mfccs, sr=real_sr, x_axis="time")
plt.colorbar()
plt.title("Real Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

In [ ]:
import librosa
import matplotlib.pyplot as plt

real_ad, real_sr = librosa.load('/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/fake/file10016.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav')
real_mfccs = librosa.feature.mfcc(y=real_ad, sr=real_sr)

plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mfccs, sr=real_sr, x_axis="time")
plt.colorbar()
plt.title("Real Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

In [ ]:
import librosa
import matplotlib.pyplot as plt

real_ad, real_sr = librosa.load('/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/fake/file10064.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav')
real_mfccs = librosa.feature.mfcc(y=real_ad, sr=real_sr)

plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mfccs, sr=real_sr, x_axis="time")
plt.colorbar()
plt.title("Real Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

In [ ]:
import librosa
import matplotlib.pyplot as plt

real_ad, real_sr = librosa.load('/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/real/file10025.wav_16k.wav_norm.wav_mono.wav_silence.wav')
real_mfccs = librosa.feature.mfcc(y=real_ad, sr=real_sr)

plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mfccs, sr=real_sr, x_axis="time")
plt.colorbar()
plt.title("Real Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

In [ ]:
import librosa
import matplotlib.pyplot as plt

real_ad, real_sr = librosa.load('/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/real/file10095.wav_16k.wav_norm.wav_mono.wav_silence.wav')
real_mfccs = librosa.feature.mfcc(y=real_ad, sr=real_sr)

plt.figure(figsize=(14, 5))
librosa.display.specshow(real_mfccs, sr=real_sr, x_axis="time")
plt.colorbar()
plt.title("Real Audio Mel-Frequency Cepstral Coefficients (MFCCs)")
plt.show()

In [ ]:
import pandas as pd

data=pd.read_excel("/kaggle/input/rerec-audio-xlsx/rerec_audio.xlsx")

***Scaling of rerec***

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Convert to DataFrame
df = pd.DataFrame(data)
df1=df.iloc[:,1:41]

# print(df1)
# Initialize MinMaxScaler with desired range (0, 1 by default)
scaler = MinMaxScaler(feature_range=(0, 1))

# Apply Min-Max normalization
normalized_data = scaler.fit_transform(df1)

# Convert the normalized data back to a DataFrame
normalized_df = pd.DataFrame(normalized_data, columns=df1.columns)
# print(df.iloc[:,41])
normalized_df["LABEL"]=df.iloc[:,41]
normalized_df.insert(0, "AUDIO", df.iloc[:,0])

print("Original Data:")
print(df)
print("\nNormalized Data:")
print(normalized_df)

In [ ]:
# import os
# import librosa
# import numpy as np
# from sklearn.preprocessing import StandardScaler
# from hashlib import md5

# # Path to your dataset
# dataset_path = "/kaggle/input/rerec-audio-xlsx/rerec_audio.xlsx"
# processed_path = "/kaggle/working/"
# os.makedirs(processed_path, exist_ok=True)

# # Initialize Standard Scaler for normalization
# scaler = StandardScaler()

# # Helper function to check 0-bit files
# for file_path in dataset_path
#     def is_zero_bit(file_path):
#         return os.path.getsize(file_path) == 0

# # Function to preprocess audio files
# def preprocess_audio_files(dataset_path, target_sample_rate=16000):
#     # Dictionary to track duplicates
#     file_hashes = set()
    
#     for root, _, files in os.walk(dataset_path):
#         for file in files:
#             file_path = os.path.join(root, file)
            
#             # Skip 0-bit files
#             if is_zero_bit(file_path):
#                 print(f"Skipping 0-bit file: {file}")
#                 continue
            
#             # Load audio file
#             try:
#                 audio, sr = librosa.load(file_path, sr=target_sample_rate)
#             except Exception as e:
#                 print(f"Error loading {file_path}: {e}")
#                 continue
            
#             # Check for duplicate files by hash
#             file_hash = md5(audio.tobytes()).hexdigest()
#             if file_hash in file_hashes:
#                 print(f"Skipping duplicate file: {file}")
#                 continue
#             file_hashes.add(file_hash)
            
#             # Zero-pad or trim to target length (16000 samples)
#             if len(audio) < target_sample_rate:
#                 audio = np.pad(audio, (0, target_sample_rate - len(audio)), mode='constant')
#             else:
#                 audio = audio[:target_sample_rate]
            
#             # Standardize audio using StandardScaler
#             audio = scaler.fit_transform(audio.reshape(-1, 1)).flatten()
            
#             # Save processed audio file
#             output_file_path = os.path.join(processed_path, file)
#             librosa.output.write_wav(output_file_path, audio, target_sample_rate)
#             print(f"Processed and saved: {output_file_path}")
# # Run the preprocessing
# preprocess_audio_files(dataset_path, processed_path)

                

In [ ]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Load your dataset (replace 'your_dataset.csv' with your dataset file)
# # Ensure that you have a column labeled 'output' or rename it accordingly
# data = pd.read_excel("/kaggle/input/rerec-audio-xlsx/rerec_audio.xlsx")

# # Check the unique categories in the output column
# output_categories = data['LABEL'].unique()

# df=pd.DataFrame(data)
# # Separate the data by categories
# for category in output_categories:
#     subset = df[df['LABEL'] == category]
    
#     # Scatter plot for two numerical columns by category
#     subset.hist(bins=20, figsize=(15, 10))

#     plt.xlabel(category)
#     plt.ylabel('Feature 2')
#     plt.tight_layout()

#     plt.show()


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jasminpradhan/2sec-audio")

print("Path to dataset files:", path)

***Scaling of 2sec***

In [ ]:
# 2sec dataset normalisation

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Convert to DataFrame
sec2_df = pd.DataFrame(data)
sec2_df1=sec2_df.iloc[:,1:41]

# print(df1)
# Initialize MinMaxScaler with desired range (0, 1 by default)
scaler = MinMaxScaler(feature_range=(0, 1))

# Apply Min-Max normalization
sec2_normalized_data = scaler.fit_transform(sec2_df1)

# Convert the normalized data back to a DataFrame
sec2_normalized_df = pd.DataFrame(sec2_normalized_data, columns=df1.columns)
# print(df.iloc[:,41])
sec2_normalized_df["LABEL"]=sec2_df.iloc[:,41]
sec2_normalized_df.insert(0, "AUDIO", sec2_df.iloc[:,0])


print("Original Data:")
print(sec2_df)
print("\nNormalized Data:")
print(sec2_normalized_df)




In [ ]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Load your dataset (replace 'your_dataset.csv' with your dataset file)
# # Ensure that you have a column labeled 'output' or rename it accordingly
# sec2_data = pd.read_excel("/kaggle/input/2sec-audio/2sec_audio_dataset.xlsx")

# # Check the unique categories in the output column
# output_categories = sec2_data['LABEL'].unique()

# sec2_df=pd.DataFrame(sec2_data)
# # Separate the data by categories
# for category in output_categories:
#     sec2_subset = sec2_df[sec2_df['LABEL'] == category]
    
#     # Scatter plot for two numerical columns by category
#     sec2_subset.hist(bins=20, figsize=(15, 10))

#     plt.xlabel(category)
#     plt.ylabel('MFCCs')
#     plt.tight_layout()

#     plt.show()
